# Uniform Cost Search

In this exercise you'll implement extend breadth-first search by incorporating a cost for each action.

In [1]:
from queue import Queue
import numpy as np
from enum import Enum

https://wiki.python.org/moin/TimeComplexity gives a solid overview of Python data structures and their time complexity. 

Is there a data structure more suitable for this task than a `Queue`?

* [`Enum`](https://docs.python.org/3/library/enum.html#module-enum) is used to represent possible actions on the grid.

In [53]:
class Action(Enum):
    """
    An action is represented by a 3 element tuple.
    
    The first 2 values are the delta of the action relative
    to the current grid position. The third and final value
    is the cost of performing the action.
    """
    UPRIGHT = (-1, 1, 2.**.5)
    DOWNRIGHT = (1, 1, 2.**.5)
    DOWNLEFT = (1, -1, 2.**.5)
    UPLEFT = (-1, -1,2.**.5)
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
    
    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'
        elif self == self.UPRIGHT:
            return "u"
        elif self == self.DOWNRIGHT:
            return "d"
        elif self == self.DOWNLEFT:
            return "/"
        elif self == self.UPLEFT:
            return "l"
    
    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])
            
    
def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and current node.
    """
    valid = [
        Action.UP, 
        Action.LEFT, 
        Action.RIGHT, 
        Action.DOWN, 
        Action.UPRIGHT, 
        Action.UPLEFT, 
        Action.DOWNRIGHT, 
        Action.DOWNLEFT,
    ]
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = current_node
    
    # check if the node is off the grid or
    # it's an obstacle
    
    if x - 1 < 0 or grid[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 > n or grid[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if y - 1 < 0 or grid[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if y + 1 > m or grid[x, y+1] == 1:
        valid.remove(Action.RIGHT)
    if y + 1 > m or x + 1 > n or grid[x+1, y+1] == 1:
        valid.remove(Action.DOWNRIGHT)
    if y - 1 > m or x - 1 < 0 or grid[x-1, y-1] == 1:
        valid.remove(Action.UPLEFT)
    if y + 1 > m or x - 1 < 0 or grid[x-1, y+1] == 1:
        valid.remove(Action.UPRIGHT)
    if y - 1 < 0 or x + 1 > n or grid[x+1, y-1] == 1:
        valid.remove(Action.DOWNLEFT)
        
    return valid

def visualize_path(grid, path, start):
    sgrid = np.zeros(np.shape(grid), dtype=np.str)
    sgrid[:] = ' '
    sgrid[grid[:] == 1] = 'O'
    
    pos = start
    
    for a in path:
        da = a.value
        sgrid[pos[0], pos[1]] = str(a)
        pos = (pos[0] + da[0], pos[1] + da[1])
    sgrid[pos[0], pos[1]] = 'G'
    sgrid[start[0], start[1]] = 'S'  
    return sgrid

### Cost Search

In this section you will extend the breadth-first search algorithm by incorporating a cost for each action. Your task is to compute the lowest cost path. Does this change the data structures you should use?

You will need to implement the remaining `TODOs`.

In [58]:
def get_path_info(branch, start, goal):
    n = goal
    path = []
    path_cost = 0 
        
    if start == goal:
        return (0, set())
    while branch[n][1] != start:
        path.append(branch[n][2])
        path_cost += branch[n][0]
        n = branch[n][1]
    return (path_cost, set(path))

# Define your breadth-first search function here
def uniform_cost(grid, start, goal):

    q = Queue()
    visited = set()
    branch = {start:None}
    found = False
    
    q.put(start)
    visited.add(start)
    
    while not q.empty(): 
        current_node = q.get()
        
        if current_node[0] == goal[0] and current_node[1] == goal[1]: 
            print('Found a path.')
            found = True
            break
        else:
            actions = valid_actions(grid, current_node)
            for action in valid_actions(grid, current_node):
                delta = action.delta
                next_cost = action.cost
                next_node = (current_node[0] + delta[0], current_node[1] + delta[1])
                
                #  if next node has been already visited then we calcualte path cose to get to that node
                #  if path cost is hight then the new one we replace it with a new one
                if next_node in visited:
                    
                    # previous path cose
                    (cost_1, path_1) = get_path_info(branch, start, next_node)
                    (current_cost, path_2) = get_path_info(branch, start, current_node)
                    cost_2 = current_cost + next_cost 

                    if cost_2 < cost_1:
                        path_difference = path_1.difference(path_2)
                        visited = visited.difference(path_difference)
                        q.put(next_node)
                        visited.add(next_node)
                        branch[next_node] = (next_cost, current_node, action)
                        
                if next_node not in visited:
                    q.put(next_node)
                    visited.add(next_node)
                    branch[next_node] = (next_cost, current_node, action)


    path = []
    path_cost = 0
    if found:
        
        # retrace steps
        path = []
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][2])
            n = branch[n][1]
            path_cost += branch[n][0]
        path.append(branch[n][2])
            
    return path[::-1], path_cost

### Executing the search

Run `uniform_cost()` and reference the grid to see if the path makes sense.

In [59]:
start = (0, 0)
goal = (4, 4)

grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0],
])

In [60]:
path, path_cost = uniform_cost(grid, start, goal)
print(path_cost, path)

Found a path.
8.242640687119286 [<Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWNRIGHT: (1, 1, 1.4142135623730951)>, <Action.RIGHT: (0, 1, 1)>, <Action.UPRIGHT: (-1, 1, 1.4142135623730951)>, <Action.DOWNRIGHT: (1, 1, 1.4142135623730951)>]


In [61]:
# S -> start, G -> goal, O -> obstacle
visualize_path(grid, path, start)

array([['S', 'O', ' ', ' ', ' ', ' '],
       ['v', 'O', ' ', 'O', ' ', ' '],
       ['v', 'O', ' ', ' ', ' ', ' '],
       ['d', 'O', ' ', 'd', 'O', ' '],
       [' ', '>', 'u', 'O', 'G', ' ']], dtype='<U1')

[Solution](/notebooks/Cost-Solution.ipynb)